In [15]:
#imports
import datetime

def time_diff_hours (date, time, start_time):
    current_time = convert_to_timeDate(date,time)
    elapsed_time = current_time - start_time
    #changing elapsed time into hours
    seconds = elapsed_time.total_seconds()
    hours = round((seconds /60 /60),2)
    return hours


def convert_to_timeDate (date, time):
    #getting the date
    first_dash = date.find("/")
    month = int(date[first_dash- 1: first_dash])
    half_date = date[first_dash + 1: len(date)]
    second_dash = half_date.find("/")
    day = int(half_date[0: second_dash])
    year = int(half_date[second_dash + 1: len(half_date)])
    #getting the time
    first_colon = time.find(":")
    initial_hour = int(time[0 : first_colon])
    half_time = time[first_colon + 1: len(time)]
    second_colon = half_time.find(":")
    minute = int(half_time[0: second_colon])
    space = half_time.find(" ")
    second = int(half_time[second_colon + 1: space])
    night = half_time[space + 1: len(half_time)]
    if night == "AM":
        if initial_hour == 12:
            hour = 0
        else:
            hour = initial_hour
    elif night == "PM":
        if initial_hour == 12:
            hour = 12
        else:
            hour = initial_hour + 12
    #get timeDate
    timeDate = datetime.datetime(year, month, day, hour, minute, second)
    return timeDate


def find_initial_time (DTlist):
    times = []
    i = 0
    for item in DTlist:
        comma = item.find(",")
        date = item[0: comma]
        time = item[comma + 2: len(item)]
        date_time = convert_to_timeDate(date, time)
        times.append(date_time)
    first = min(times)
    return first

2019-08-06 12:48:16
